In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import emoji
from nltk import word_tokenize
from string import punctuation, ascii_letters
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from gensim.models import Word2Vec

In [2]:
MAX_WORDS = 2000
MAX_LEN = 30
EMB_SIZE = 100

In [3]:
morph = MorphAnalyzer(lang='ru')

In [4]:
stop_words = set(get_stop_words('ru'))

In [5]:
punctuation = set(punctuation).union((' ', '«', '»', '—', '–', '“', '”', '…'))

In [6]:
cyrillic_letters = set([chr(i) for i in range(ord('а'), ord('я') + 1)] +
                       [chr(i) for i in range(ord('А'), ord('Я') + 1)] +
                       ['ё', 'Ё'])

In [7]:
data = pd.read_excel('../data/summer_reviews.xls')
data.columns = ['rating', 'content', 'date']

In [8]:
data['target'] = (data.rating > 3).astype(int)

Попробуем опять же учитывать эмоджи.

In [9]:
def preprocess(text):
    tokens = word_tokenize(text)
    result = []
    for token in tokens:
        if (set(token).intersection(cyrillic_letters)
            or set(token).intersection(set(ascii_letters))
            or token in emoji.UNICODE_EMOJI):
            result.append(token)
    tokens = [token.lower() for token in result if token.lower() not in stop_words]
    tokens = [token for token in tokens if token in emoji.UNICODE_EMOJI or len(token) >  1]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens
        

In [10]:
data['tokens'] = data.content.apply(lambda x: preprocess(str(x)))

In [11]:
tokens_freq = Counter([val for sublist in data.tokens.tolist() for val in sublist]).most_common(MAX_WORDS)
tokens_freq = [word for word, _ in tokens_freq]

In [12]:
vocab = {v: k for k, v in enumerate(tokens_freq, start=2)}
vocab['UNK'] = 1
vocab['PAD'] = 0

In [14]:
def tokens_to_indices(tokens):
    return [vocab.get(token, 1) for token in tokens]

In [15]:
data['indices'] = data.tokens.apply(lambda x: tokens_to_indices(x))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data['indices'].values,
                                                    data['target'].values,
                                                    test_size=0.1,
                                                    random_state=42)

In [17]:
X_train = pad_sequences(X_train, MAX_LEN, padding='post', truncating='post')
X_test = pad_sequences(X_test, MAX_LEN, padding='post', truncating='post')

Обучаем эмбеддинги внутри сети.

In [27]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab), output_dim=EMB_SIZE, activity_regularizer=regularizers.l2(1e-6))(inputs)
conv_1 = tf.keras.layers.Conv1D(kernel_size=3, filters=EMB_SIZE, strides=1, activation='relu')(embeddings)
pool_1 = tf.keras.layers.MaxPooling1D()(conv_1)
conv_2 = tf.keras.layers.Conv1D(kernel_size=2, filters=int(EMB_SIZE / 2), strides=1, activation='relu')(pool_1)
pool_2 = tf.keras.layers.MaxPooling1D()(conv_2)
conv_3 = tf.keras.layers.Conv1D(kernel_size=2, filters=int(EMB_SIZE / 2), strides=1, activation='relu')(pool_2)
pool_3 = tf.keras.layers.MaxPooling1D()(conv_3)
flat = tf.keras.layers.Flatten()(pool_3)
dense_1 = tf.keras.layers.Dense(64, activation='relu', activity_regularizer=regularizers.l2(1e-4))(flat)
drop_1 = tf.keras.layers.Dropout(0.2)(dense_1)
dense_2 = tf.keras.layers.Dense(32, activation='relu', activity_regularizer=regularizers.l2(1e-4))(drop_1)
drop_2 = tf.keras.layers.Dropout(0.2)(dense_2)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense_2)

In [28]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [30]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [31]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          batch_size=256,
          epochs=20)

Train on 18593 samples, validate on 2066 samples
Epoch 1/20
18593/18593 [==============================] - 2s 106us/sample - loss: 0.4078 - accuracy: 0.8369 - val_loss: 0.2985 - val_accuracy: 0.8698
Epoch 2/20
18593/18593 [==============================] - 1s 75us/sample - loss: 0.2400 - accuracy: 0.8991 - val_loss: 0.2343 - val_accuracy: 0.9042
Epoch 3/20
18593/18593 [==============================] - 1s 76us/sample - loss: 0.1941 - accuracy: 0.9230 - val_loss: 0.2189 - val_accuracy: 0.9071
Epoch 4/20
18593/18593 [==============================] - 1s 74us/sample - loss: 0.1707 - accuracy: 0.9329 - val_loss: 0.2373 - val_accuracy: 0.9042
Epoch 5/20
18593/18593 [==============================] - 1s 76us/sample - loss: 0.1506 - accuracy: 0.9430 - val_loss: 0.2586 - val_accuracy: 0.9013
Epoch 6/20
18593/18593 [==============================] - 1s 76us/sample - loss: 0.1303 - accuracy: 0.9554 - val_loss: 0.2710 - val_accuracy: 0.9042
Epoch 7/20
18593/18593 [==============================] 

Обучаем эмбеддинги отдельно.

In [32]:
w2v = Word2Vec(sentences=data.tokens.values, size=EMB_SIZE, min_count=3, window=3, workers=-1, sg=1)

In [33]:
tokens_freq = Counter([val for sublist in data.tokens.tolist() for val in sublist]).most_common()
tokens_freq = [word for word, _ in tokens_freq]

In [34]:
vocab = {v: k for k, v in enumerate(tokens_freq, start=2)}
vocab['UNK'] = 1
vocab['PAD'] = 0

In [35]:
def get_weights(vocab, model):

    dim = model.vector_size
    weights = np.zeros((len(vocab), dim))

    for word, i in vocab.items():
        if word == 'PAD':
            continue
        if word == 'UNK':
            weights[i] = np.random.normal(0, 2, dim)
        try:
            weights[i] = model.wv.get_vector(word)
        except KeyError:
            weights[i] = np.random.normal(0, 2, dim)
    return weights

In [36]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       weights=[get_weights(vocab, w2v)],
                                       trainable=False,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
conv_1 = tf.keras.layers.Conv1D(kernel_size=3, filters=EMB_SIZE, strides=1, activation='relu')(embeddings)
pool_1 = tf.keras.layers.MaxPooling1D()(conv_1)
conv_2 = tf.keras.layers.Conv1D(kernel_size=2, filters=int(EMB_SIZE / 2), strides=1, activation='relu')(pool_1)
pool_2 = tf.keras.layers.MaxPooling1D()(conv_2)
conv_3 = tf.keras.layers.Conv1D(kernel_size=2, filters=int(EMB_SIZE / 2), strides=1, activation='relu')(pool_2)
pool_3 = tf.keras.layers.MaxPooling1D()(conv_3)
flat = tf.keras.layers.Flatten()(pool_3)
dense_1 = tf.keras.layers.Dense(64, activation='relu', activity_regularizer=regularizers.l2(1e-4))(flat)
drop_1 = tf.keras.layers.Dropout(0.2)(dense_1)
dense_2 = tf.keras.layers.Dense(32, activation='relu', activity_regularizer=regularizers.l2(1e-4))(drop_1)
drop_2 = tf.keras.layers.Dropout(0.2)(dense_2)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense_2)

In [37]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [38]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [39]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [40]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          batch_size=256,
          epochs=30)

Train on 18593 samples, validate on 2066 samples
Epoch 1/30
18593/18593 [==============================] - 2s 85us/sample - loss: 0.5050 - accuracy: 0.8267 - val_loss: 0.4210 - val_accuracy: 0.8335
Epoch 2/30
18593/18593 [==============================] - 1s 57us/sample - loss: 0.4207 - accuracy: 0.8342 - val_loss: 0.4205 - val_accuracy: 0.8364
Epoch 3/30
18593/18593 [==============================] - 1s 56us/sample - loss: 0.4171 - accuracy: 0.8367 - val_loss: 0.4184 - val_accuracy: 0.8345
Epoch 4/30
18593/18593 [==============================] - 1s 57us/sample - loss: 0.4100 - accuracy: 0.8368 - val_loss: 0.4055 - val_accuracy: 0.8349
Epoch 5/30
18593/18593 [==============================] - 1s 57us/sample - loss: 0.3745 - accuracy: 0.8413 - val_loss: 0.3718 - val_accuracy: 0.8379
Epoch 6/30
18593/18593 [==============================] - 1s 56us/sample - loss: 0.3474 - accuracy: 0.8505 - val_loss: 0.3554 - val_accuracy: 0.8393
Epoch 7/30
18593/18593 [==============================] -

В принципе результат на валидационных данных примерно такой же или чуть хуже.